In [2]:
import numpy as np

from qiskit import Aer, transpile, assemble
backend = Aer.get_backend("qasm_simulator")
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, visualization
from qiskit.algorithms.optimizers import COBYLA
# Initialize the COBYLA optimizer
optimizer=COBYLA(maxiter=100, tol=0.0001)

import matplotlib.pyplot as plt
%matplotlib inline


NUM_SHOTS=10000
np.random.seed(999999)


episodes=10

t=10    #
to=2
T=0


# the below is currently not used as it will complicate things further

N=4      # qubits 
l=1      # layers
k=1      # gates per layer
m=N-1    # CNOT entaglement gates per layer

c=0.001   # to prevent the taking the (undefined) log of zero




def get_var_form(params):
    
    
    qr = QuantumRegister(N, name="q")
    cr = ClassicalRegister(N, name='c')
    qc = QuantumCircuit(qr, cr)
    
    
# this is the entangling circuit, a reversed GHZ

    for i in range(N-1):
        qc.cx(0,i+1)
    qc.h(0)
    
    
    
    for i in range(N):
        qc.rz(params[i], qr[i])
        
        # qx.rz works (wrong orintation) but qc.ry causes errors - why is that? The H-gate is a pi/2 rotation of the RY gate
        # can you figure out where the problem lies here 

        # in the proper implementation we will have U gates with 3 parameters
        
        
        qc.cx(0,N-1)
        
          
    qc.measure_all()
    
    return qc

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return np.asarray(output_distr)


def ProbDist(params):
    
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    t_qc = transpile(qc, backend)
    qobj = assemble(t_qc, shots=NUM_SHOTS)
    result = backend.run(qobj).result()
    
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    return get_probability_distribution(result.get_counts(qc))
    


def objective_function(params):
    
   
    paramsCopy=params
    totalCFI=0
    
    
    for i in range(N):
        
        # calculate probability distributions separately for each parameter derivative
        # using parameter shift rule with theta-pi/2 and theta+pi/2
        
        params=paramsCopy
        pr=ProbDist(params)
        
        params[i]=params[i]-np.pi/2       # parameter shift rule 
        prMin=ProbDist(params)
        
        params=paramsCopy
        params[i]=params[i]+np.pi/2
        prPlus=ProbDist(params)
        
        # calculate Fisher Information
        # CFI(w)= E( dlog(Pr(X|w)/dw)^2 )
        # this is equivalent (chainrule) to E( (1/Pr(X|w) * d(Pr(X|w)/dw )^2)
        # we will run circuit with both versions as the parameter shift rule strictly speaking
        # would only apply to dPr(X|w)/dw not the log of it
        # the goal is to maximise the Fisher Information to reduce the signal noise ratio
        # the full formula is CFI*t^2/(t+t_overhead)*T
        # however t and T are just factors and should not have much impact on the optimization process
        # like wise the 1/2 multiplicator for the parameter shift rule should make little difference
    
        totalCFI=np.sum(np.square((np.log(prPlus+c)-np.log(prMin+c))))+totalCFI
        
        # in principle the above term should have been divided by the number of entries
        # but this is not required as it would be the same for all distributions
        # for the same reason we do not need to use the full formula but 
        # just try to maximise CFI
    
    
    # the Fisher Information is the cost
    
    # COBYLA is a minimizer; therefore we use the negative value of the CFI to maximize 
    
    return -totalCFI/N     #returning cost
    
    
    


# intialise circuit with random theta values

params = np.random.rand(N)*2*np.pi

# maximising Fisher Information with COBYLA  

    
result = optimizer.minimize( fun=objective_function,x0=params)

print(result)    

qc = get_var_form(result.x)
counts = backend.run(qc, shots=10000).result().get_counts()


print("Parameters Found:", result.x)
print("Cost:", objective_function(result.x))




       



    
    



{   'fun': -0.00010772592484429562,
    'jac': None,
    'nfev': 45,
    'nit': None,
    'njev': None,
    'x': array([2.94964406, 1.82998269, 5.45771748, 0.49408595])}
Parameters Found: [2.94964406 1.82998269 5.45771748 0.49408595]
Cost: -0.0003057231285929195
